In [7]:
import pandas as pd
import zipfile

# Extract and read the dataset
with zipfile.ZipFile('/mnt/data/dataset and all.zip', 'r') as zip_ref:
    zip_ref.extractall('/mnt/data/')

# Assuming the dataset is in a CSV file within the extracted content
df = pd.read_csv('/mnt/data/dataset.csv')

# Display the first few rows of the dataframe
print(df.head())



FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/dataset and all.zip'

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Separating features and target variables
X = df.drop(columns=['xyz_vaccine', 'seasonal_vaccine', 'respondent_id'])
y = df[['xyz_vaccine', 'seasonal_vaccine']]

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identifying categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combining preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)])

# Example model (Random Forest)
from sklearn.ensemble import RandomForestClassifier

# Creating the model pipeline
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))])

# Training the model
model.fit(X_train, y_train)


NameError: name 'df' is not defined

In [11]:
from sklearn.metrics import roc_auc_score

# Making predictions
y_pred_proba = model.predict_proba(X_test)

# Extracting probabilities for each target variable
xyz_vaccine_pred = y_pred_proba[0][:, 1]  # Probability of class 1 for xyz_vaccine
seasonal_vaccine_pred = y_pred_proba[1][:, 1]  # Probability of class 1 for seasonal_vaccine

# Calculating ROC AUC scores
roc_auc_xyz = roc_auc_score(y_test['xyz_vaccine'], xyz_vaccine_pred)
roc_auc_seasonal = roc_auc_score(y_test['seasonal_vaccine'], seasonal_vaccine_pred)

# Mean ROC AUC score
mean_roc_auc = (roc_auc_xyz + roc_auc_seasonal) / 2

print(f'ROC AUC for xyz_vaccine: {roc_auc_xyz}')
print(f'ROC AUC for seasonal_vaccine: {roc_auc_seasonal}')
print(f'Mean ROC AUC: {mean_roc_auc}')


NameError: name 'model' is not defined